# NLLB-1.3B Model Exploration

**For Google Colab:**
1. Mount Google Drive (run cell below)
2. Set `ROOT_DIR` to your project folder path
3. Run the rest of the notebook

**For local execution:** Skip the Google Drive cell and run from "Import Libraries"

In [ ]:
# Mount Google Drive (only needed for Google Colab)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # IMPORTANT: Set this to your code_fr_en directory path
    # This should point to where THIS notebook is located
    ROOT_DIR = "/content/drive/MyDrive/UofT/CSC2517/term_paper/code_fr_en"
    
    import os
    os.chdir(ROOT_DIR)
    print(f"✓ Changed to: {os.getcwd()}")
except ImportError:
    print("Not running on Colab, using local environment")

In [ ]:
# Verify working directory and model files
import os
print(f"Current directory: {os.getcwd()}")
print(f"\nContents of current directory:")
print(os.listdir('.'))

model_path = "../models/nllb-1.3B"
if os.path.exists(model_path):
    print(f"\n✓ Model directory exists: {model_path}")
else:
    print(f"\n✗ Model directory NOT found: {model_path}")
    print("\nMake sure you:")
    print("1. Ran 03_load_model.py to download the model")
    print("2. Set ROOT_DIR to point to code_fr_en/ directory")
    print(f"\nExpected structure:")
    print(f"  ROOT_DIR/")
    print(f"  ├── models/")
    print(f"  │   └── nllb-1.3B/")
    print(f"  └── <you are here>")

## Import Libraries

In [ ]:
# Import all required libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from datasets import load_from_disk

## 1. Load the saved model

In [ ]:
# Load tokenizer and model from local directory
model_dir = "../models/nllb-1.3B"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_dir)
print("✓ Tokenizer loaded")

print("\nLoading model...")
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
print("✓ Model loaded")

# Determine device: CUDA (Colab/NVIDIA) > MPS (Apple Silicon) > CPU
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

model = model.to(device)
print(f"\n✓ Model moved to device: {device}")

print(f"\nModel: NLLB-200-distilled-1.3B")
print(f"Parameters: ~1.3B")

## 2. Test English → French translation

In [ ]:
# NLLB uses language codes: eng_Latn (English), fra_Latn (French)
test_sentence = "Hello, how are you?"
print(f"Input (English): {test_sentence}")

# Set source language
tokenizer.src_lang = "eng_Latn"

# Tokenize and move to device
inputs = tokenizer(test_sentence, return_tensors="pt").to(device)

# Generate translation
translated_tokens = model.generate(
    **inputs,
    forced_bos_token_id=tokenizer.convert_tokens_to_ids("fra_Latn"),
    max_length=50
)

# Decode
translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
print(f"Output (French): {translation}")

## 3. Test with multiple sentences

In [ ]:
test_sentences = [
    "Hello, how are you?",
    "I am a student at the university.",
    "The cat is on the table.",
    "What time is it?",
    "I love learning languages."
]

print("="*80)
print("English → French Translations")
print("="*80)

tokenizer.src_lang = "eng_Latn"

for i, sentence in enumerate(test_sentences, 1):
    inputs = tokenizer(sentence, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids("fra_Latn"),
        max_length=50
    )
    translation = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    print(f"\n{i}. EN: {sentence}")
    print(f"   FR: {translation}")

## 4. Inspect model architecture

In [ ]:
# Check model configuration
print("Model Configuration:")
print(f"  Number of encoder layers: {model.config.encoder_layers}")
print(f"  Number of decoder layers: {model.config.decoder_layers}")
print(f"  Number of attention heads: {model.config.encoder_attention_heads}")
print(f"  Hidden size: {model.config.d_model}")
print(f"  Vocabulary size: {model.config.vocab_size}")
print(f"\nModel has encoder-decoder architecture for sequence-to-sequence translation")

In [ ]:
# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"\nParameters (in millions): {total_params / 1e6:.1f}M")

## 5. Test with dataset examples

In [ ]:
# Load some examples from our saved dataset
dataset = load_from_disk("../data/wmt14_fr_en_validation_2000")
print(f"Loaded {len(dataset)} sentence pairs\n")

# Test on first 3 examples
print("="*80)
print("Testing on WMT14 dataset examples (English → French)")
print("="*80)

tokenizer.src_lang = "eng_Latn"

for i in range(3):
    example = dataset[i]["translation"]
    english = example["en"]
    french_ref = example["fr"]
    
    # Translate
    inputs = tokenizer(english, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids("fra_Latn"),
        max_length=100
    )
    translation = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    
    print(f"\nExample {i+1}:")
    print(f"EN: {english}")
    print(f"FR (reference): {french_ref}")
    print(f"FR (translated): {translation}")

## Summary

**Model loaded successfully:**
- NLLB-200-distilled-1.3B (~1.3B parameters)
- English → French translation working
- GPU acceleration enabled (CUDA/MPS/CPU)
- Ready for attention extraction

**Next steps:**
1. Extract attention weights from encoder and decoder
2. Build attention graphs
3. Compute persistent homology